In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('2009.csv')
y = df.DEP_DELAY.replace(np.nan, 0)
X = df[["OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEP_TIME","DEP_TIME","TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN","CRS_ARR_TIME","ARR_TIME","ARR_DELAY"]].replace(np.nan, 0)

In [6]:
def convert_to_int(df, col):
    df[col] = df[col].apply(lambda x: sum(ord(c) for c in x))
str_cols = ["OP_CARRIER", "ORIGIN", "DEST"]
for col in str_cols: 
  convert_to_int(X, col)
X = np.asarray(X).astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error as mae
def predictFlights(record_data, delay_times):
    print(record_data.shape)
    print(delay_times.shape)
    # define features and outcome variable
    # initialize k folds
    cv = KFold(n_splits=5, shuffle=True)
    actual_results = []
    prediction_results = []

    model = LinearRegression()
    # perform cross validation
    for train_index, test_index in cv.split(record_data):
        # fit the model on the training data
        model.fit(X_train, y_train)
        # base predictions on the testing data 
        predictions = model.predict(X_test)
        actual = y_test.values.tolist()
        # append the prediction results and actual results to a list for analysis after the cross validation
        prediction_results.append(predictions.tolist())
        actual_results.append(actual)

    actual_results = sum(actual_results, [])
    prediction_results = sum(prediction_results, [])
    error = mae(actual_results, prediction_results)
    print(f'The mean absolute error across all folds is: {error}')
    print('\n')
    return model
model = predictFlights(X, y)

(6429338, 13)
(6429338,)
The mean absolute error across all folds is: 7.124943682833761




In [4]:
import pickle
model_file = 'prediction_model.sav'
pickle.dump(model, open(model_file, 'wb'))

In [ ]:
entry_1 = X.iloc[0:10][["OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEP_TIME","DEP_TIME","TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN","CRS_ARR_TIME","ARR_TIME","ARR_DELAY","CANCELLED","CANCELLATION_CODE","DIVERTED","CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME","AIR_TIME","DISTANCE","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]].replace(np.nan, 0)
model.predict(entry_1)

In [ ]:
from keras.utils import to_categorical
import tensorflow as tf
from keras.layers import Dropout
model = tf.keras.Sequential([tf.keras.layers.Dense(512, input_shape=(25,), activation='relu'),
                    tf.keras.layers.Dense(128, activation='relu'),
                    tf.keras.layers.Dense(512, activation='relu'),
                    tf.keras.layers.Dense(128, activation='relu'),
                    tf.keras.layers.Dense(100, activation='relu'),
                    tf.keras.layers.Dense(512, activation='relu'),
                    tf.keras.layers.Dense(128, activation='relu'),
                    tf.keras.layers.Dense(10, activation='softmax')])

# Compile model
model.compile(optimizer='Adam', loss='mean_squared_error',metrics=['accuracy'],)
print("Train image shape: ", X_train.shape)
print(y_train.shape)

print(X_train.shape)
# Train model
model.summary()
model.fit(np.array(X_train), np.array(y_train), epochs=3000, validation_split=0.2)